### Extending the SNIPS dataset using Seamless M4T-V2 model

In [1]:
from seamless_communication.inference import Translator
import torch
import pandas as pd
from datasets import Dataset,DatasetDict
# from seamless_communication.streaming.dataloaders.s2tt import SileroVADSilenceRemover

2024-05-12 18:02:31,964 INFO -- numexpr.utils: Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-05-12 18:02:31,964 INFO -- numexpr.utils: NumExpr defaulting to 8 threads.
2024-05-12 18:02:32,682 INFO -- datasets: PyTorch version 2.1.1 available.
2024-05-12 18:02:32,684 INFO -- datasets: TensorFlow version 2.15.0.post1 available.


In [143]:
from datasets import load_dataset,DatasetDict
dataset = load_dataset('benayas/snips')
train_valid = dataset['train'].train_test_split(test_size=0.1)

dataset_snips = DatasetDict({
    'train': train_valid['train'],
    'test': dataset['test'],
    'validation': train_valid['test']})

In [144]:
dataset_snips

DatasetDict({
    train: Dataset({
        features: ['text', 'category'],
        num_rows: 11775
    })
    test: Dataset({
        features: ['text', 'category'],
        num_rows: 1400
    })
    validation: Dataset({
        features: ['text', 'category'],
        num_rows: 1309
    })
})

In [148]:


# Define the label-to-ID mapping
label2id = {
    "AddToPlaylist": 0,
    'BookRestaurant': 1,
    'GetWeather': 2,
    'PlayMusic': 3,
    'RateBook': 4,
    'SearchCreativeWork': 5,
    'SearchScreeningEvent': 6,
}
# Define a function to add the 'label' column
def add_label_column(example):
    example['label'] = label2id[example['category']]
    return example

# Apply the function using dataset.map for each split
dataset = dataset_snips.map(add_label_column, )
# dataset = dataset.map(add_label_column, split='test')
# dataset = dataset.map(add_label_column, split='validation')

Map:   0%|          | 0/11775 [00:00<?, ? examples/s]

Map:   0%|          | 0/1309 [00:00<?, ? examples/s]

In [155]:
from datasets import load_dataset,DatasetDict,load_from_disk
dataset = load_from_disk("/snips_dataset/snips_de.hf") 
dataset['train'][1]

{'text': 'setze United Abominations auf meine seltene Groove-Playlist',
 'category': 'AddToPlaylist'}

In [2]:

# Initialize a Translator object with a multitask model, vocoder on the GPU.

model_name = "seamlessM4T_v2_large"
vocoder_name = "vocoder_v2" if model_name == "seamlessM4T_v2_large" else "vocoder_36langs"

translator = Translator(
    model_name,
    vocoder_name,
    device=torch.device("cpu"),
    dtype=torch.float16,
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 8.45G/8.45G [02:49<00:00, 53.5MB/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████| 360k/360k [00:00<00:00, 20.5MB/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████| 4.93M/4.93M [00:00<00:00, 8.00MB/s]
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 160M/160M [00:00<00:00, 252MB/s]
/home/s6amalia/anaconda3/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
from datasets import load_dataset
snips_ds = load_dataset("benayas/snips")
snips_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'category'],
        num_rows: 13084
    })
    test: Dataset({
        features: ['text', 'category'],
        num_rows: 1400
    })
})

In [4]:
features_ds=snips_ds['train'].features
features_ds

{'text': Value(dtype='string', id=None),
 'category': Value(dtype='string', id=None)}

In [5]:
from tqdm import tqdm
translated_ds_train = []

for i in tqdm(range(len(snips_ds['train']))):
    text_output, speech_output = translator.predict(
                                 input=snips_ds['train']['text'][i],
                                 task_str="t2tt",
                                 tgt_lang="tur",
                                 src_lang="eng",
                                 )
    translated_ds_train.append(text_output[0])
# print(f"Translated text in {tgt_lang}: {text_output[0]}")
# print()


  0%|                                                                                             | 4/13084 [00:16<14:34:44,  4.01s/it]

KeyboardInterrupt



In [ ]:
translated_ds_test = []

for i in tqdm(range(len(snips_ds['test']))):
    text_output, speech_output = translator.predict(
                                 input=snips_ds['test']['text'][i],
                                 task_str="t2tt",
                                 tgt_lang="hin",
                                 src_lang="eng",
                                 )
    translated_ds_test.append(text_output[0])

In [119]:
str(translated_ds_test[16]).encode('utf-8')

b'Ajoutez Blue Feather \xc3\xa0 la liste de lecture de Nature Noise'

In [135]:
def create_dataset(translated_test):
    translated_test = [str(text) for text in translated_test]
    if len(translated_test)<2000:
        df_test=pd.DataFrame({'text':translated_test, 'category':snips_ds['test']['category'] })
    else:
        df_test=pd.DataFrame({'text':translated_test, 'category':snips_ds['train']['category'] })    
    df_test['text'] = df_test['text'].str.replace('  ', '')
    df_test['text'] = df_test['text'].str.replace('⁇', '')
    df_test['text'] = df_test['text'].str.replace('¿', '')
    ds_test = Dataset.from_pandas(df_test , features = snips_ds['train'].features,preserve_index=False)
    return ds_test

In [136]:
de_test =create_dataset(translated_ds_test)
de_train =create_dataset(translated_ds_train)

In [138]:
de_test['text']

['एस्टेले मेरी वसंत प्लेलिस्ट पर होना चाहिए',
 'क्या जोविनो सैंटोस नेटो के एल्बम को मेरी कॉन्फिडेंस बूस्ट प्लेलिस्ट में जोड़ा जा सकता है?',
 'Ila की प्लेलिस्ट में जोड़ें A Mi Manera Recopilatorio नाम मेरा दिल अभी भी खड़ा था',
 'सैम स्पैरो को मेरी प्लेलिस्ट में जोड़ें जिसे बीच वाइब्स कहा जाता है',
 'कृपया इस गाने को मेरी अर्बन हिट्स प्लेलिस्ट में डालें।',
 'जोड़ें अरे, जॉनी कोप, क्या आप अभी तक मेटल 2016 प्लेलिस्ट में वर्ष के लिए जाग रहे हैं',
 'इस ट्रैक को मेरी रॉक हार्ड प्लेलिस्ट में जोड़ें',
 'Sin Ti प्लेलिस्ट में खोज और विनाश जोड़ें',
 'मैं चाहता हूँ कि आप इस कलाकार को मेरी शाम की यात्रा प्लेलिस्ट में डालें।',
 'मेरे सभी बाहर 90 के दशक प्लेलिस्ट के लिए डेव Bickler जोड़ें',
 'मेरे देश हिट्स प्लेलिस्ट में जेफ Burrows एल्बम जोड़ें',
 'मेरे शास्त्रीय नई रिलीज़ स्पॉटिफाई पिक प्लेलिस्ट में फ्रांसिस्को डी ग्रेगोरी जोड़ें',
 'मेरे लिए उसे प्यार करने के लिए इस्तेमाल किया खेल यह पशु सामूहिक प्लेलिस्ट है',
 'रात और दिन को मेरी आलसी ठंडी दोपहर की प्लेलिस्ट में जोड़ें',
 'मेरी प्लेलिस्ट में जॉनी 

In [139]:
import datasets
snips_de_dataset = datasets.DatasetDict({"train":de_train,"test":de_test})

In [140]:
snips_de_dataset.save_to_disk("snips_dataset/snips_hi.hf")

Saving the dataset (0/1 shards):   0%|          | 0/13084 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1400 [00:00<?, ? examples/s]

In [132]:
snips_de_dataset['test']['text']

['Estelle ilkbahar çalma listemde olmalı.',
 "Jovino Santos Neto'nun albümü Confidence Boost çalma listeme eklenebilir mi?",
 "Ila'nın çalma listesine ekle A Mi Manera Recopilatorio adı kalbim durdu",
 "Sam Sparro'yu Beach Vibes adlı çalma listeme ekle",
 'Lütfen bu şarkıyı Urban Hits çalma listeme ekle.',
 "Hey, Johnnie Cope, Are Ye Waking Yet'i metal 2016 çalma listesine ekle",
 'Bu şarkıyı Rock Hard çalma listeme ekle',
 'Sin Ti çalma listesine arama ve yok etme ekle',
 'Bu sanatçıyı Akşam Yolculuğu çalma listeme koymanızı istiyorum.',
 'All out 90s çalma listeme Dave Bickler  ı ekle',
 'Country Hits çalma listeme Jeff Burrows albümünü ekle',
 "Francesco De Gregori'yi klasik yeni sürümlerime ekle Spotify'ın seçtiği çalma listesi",
 'Bu hayvan toplu çalma listesi',
 'Lazy Chill Afternoon çalma listeme gece ve gündüz ekle',
 "Jonny Buckland'ı haftalık çalma listeme ekle",
 'Composer Weekly Pauline Oliveros çalma listesine parça ekle',
 'Blue Feather  ı Nature Noise çalma listesine ekl